In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
# Good data
df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [3]:
# Messy data
df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [4]:
df = df.dropna()

### Individual Classifiers are LSTMS

In [6]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = train['Input.deception_quadrant_cat'].tolist()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = test['Input.deception_quadrant_cat'].tolist()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train_col = train['Input.full_text']

X_test_col = test['Input.full_text']

In [7]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [8]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [9]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [10]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

### Construct individual LSTM models

In [11]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
85/85 [==============================] - 3s 41ms/step - loss: 0.6625 - accuracy: 0.6235 - f1_m: 0.7635 - recall_m: 0.9901 - precision_m: 0.6239 - val_loss: 0.6685 - val_accuracy: 0.6099 - val_f1_m: 0.7606 - val_recall_m: 1.0000 - val_precision_m: 0.6147
Epoch 2/15
85/85 [==============================] - 3s 35ms/step - loss: 0.6438 - accuracy: 0.6271 - f1_m: 0.7691 - recall_m: 0.9975 - precision_m: 0.6268 - val_loss: 0.6770 - val_accuracy: 0.5965 - val_f1_m: 0.7413 - val_recall_m: 0.9330 - val_precision_m: 0.6158


In [12]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [13]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
85/85 [==============================] - 4s 44ms/step - loss: 0.5877 - accuracy: 0.7351 - f1_m: 0.8429 - recall_m: 0.9901 - precision_m: 0.7392 - val_loss: 0.5985 - val_accuracy: 0.7210 - val_f1_m: 0.8373 - val_recall_m: 1.0000 - val_precision_m: 0.7213
Epoch 2/15
85/85 [==============================] - 3s 35ms/step - loss: 0.5687 - accuracy: 0.7392 - f1_m: 0.8498 - recall_m: 1.0000 - precision_m: 0.7396 - val_loss: 0.5999 - val_accuracy: 0.7210 - val_f1_m: 0.8373 - val_recall_m: 1.0000 - val_precision_m: 0.7213


In [14]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [15]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
85/85 [==============================] - 4s 42ms/step - loss: 0.5253 - accuracy: 0.7879 - f1_m: 0.8787 - recall_m: 0.9921 - precision_m: 0.7920 - val_loss: 0.5375 - val_accuracy: 0.7707 - val_f1_m: 0.8713 - val_recall_m: 1.0000 - val_precision_m: 0.7726
Epoch 2/15
85/85 [==============================] - 3s 36ms/step - loss: 0.5083 - accuracy: 0.7926 - f1_m: 0.8840 - recall_m: 1.0000 - precision_m: 0.7929 - val_loss: 0.5408 - val_accuracy: 0.7707 - val_f1_m: 0.8713 - val_recall_m: 1.0000 - val_precision_m: 0.7726


In [16]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [17]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
85/85 [==============================] - 4s 44ms/step - loss: 0.6822 - accuracy: 0.5553 - f1_m: 0.6526 - recall_m: 0.8274 - precision_m: 0.5636 - val_loss: 0.6879 - val_accuracy: 0.5413 - val_f1_m: 0.3637 - val_recall_m: 0.2529 - val_precision_m: 0.6665
Epoch 2/15
85/85 [==============================] - 3s 35ms/step - loss: 0.6370 - accuracy: 0.6272 - f1_m: 0.6263 - recall_m: 0.6014 - precision_m: 0.6729 - val_loss: 0.6884 - val_accuracy: 0.5606 - val_f1_m: 0.5407 - val_recall_m: 0.5017 - val_precision_m: 0.5927


In [18]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [19]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
85/85 [==============================] - 3s 41ms/step - loss: 0.2250 - accuracy: 0.9486 - f1_m: 0.9737 - recall_m: 0.9982 - precision_m: 0.9507 - val_loss: 0.2368 - val_accuracy: 0.9429 - val_f1_m: 0.9698 - val_recall_m: 1.0000 - val_precision_m: 0.9415
Epoch 2/15
85/85 [==============================] - 3s 35ms/step - loss: 0.2001 - accuracy: 0.9503 - f1_m: 0.9743 - recall_m: 1.0000 - precision_m: 0.9502 - val_loss: 0.2210 - val_accuracy: 0.9429 - val_f1_m: 0.9698 - val_recall_m: 1.0000 - val_precision_m: 0.9415
Epoch 3/15
85/85 [==============================] - 3s 35ms/step - loss: 0.1909 - accuracy: 0.9503 - f1_m: 0.9746 - recall_m: 1.0000 - precision_m: 0.9505 - val_loss: 0.2319 - val_accuracy: 0.9429 - val_f1_m: 0.9698 - val_recall_m: 1.0000 - val_precision_m: 0.9415


### One hot encodings

In [20]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred[i][0]
    pred_obj['reasoning'] = reasoning_pred[i][0]
    pred_obj['shareinfo'] = shareinfo_pred[i][0]
    
    pred_df_arr_full.append(pred_obj)
    
    pred_obj['rapport'] = rapport_pred[i][0]
    pred_df_arr.append(pred_obj)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [21]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred_test[i][0]
    pred_obj['reasoning'] = reasoning_pred_test[i][0]
    pred_obj['shareinfo'] = shareinfo_pred_test[i][0]
    
    pred_test_df_arr_full.append(pred_obj)
    
    pred_obj['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr.append(pred_obj)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [22]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [30]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = create_joint_model(pred_df_full)
history = joint_full_model.fit(x=pred_df_full, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df_full,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception
Epoch 1/32
169/169 [==============================] - 1s 5ms/step - loss: 0.7800 - acc: 0.0856 - f1_m: 0.9743 - precision_m: 0.9503 - recall_m: 1.0000 - val_loss: 0.6990 - val_acc: 0.2238 - val_f1_m: 0.9701 - val_precision_m: 0.9423 - val_recall_m: 1.0000
Epoch 2/32
169/169 [==============================] - 1s 3ms/step - loss: 0.6950 - acc: 0.3777 - f1_m: 0.9743 - precision_m: 0.9503 - recall_m: 1.0000 - val_loss: 0.6936 - val_acc: 0.5057 - val_f1_m: 0.9701 - val_precision_m: 0.9423 - val_recall_m: 1.0000
Epoch 3/32
169/169 [==============================] - 1s 3ms/step - loss: 0.6935 - acc: 0.5006 - f1_m: 0.9743 - precision_m: 0.9503 - recall_m: 1.0000 - val_loss: 0.6933 - val_acc: 0.5128 - val_f1_m: 0.9701 - val_precision_m: 0.9423 - val_recall_m: 1.0000
Epoch 4/32
169/169 [==============================] - 1s 3ms/step - loss: 0.6933 - acc: 0.4993 - f1_m: 0.9743 - precision_m: 0.9503 - recall_m: 1.0000 - val_loss: 0.6932 -

In [31]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = create_joint_model(pred_df)
history = joint_full_model.fit(x=pred_df, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport
Epoch 1/32
169/169 [==============================] - 1s 5ms/step - loss: 0.6935 - acc: 0.4540 - f1_m: 0.7692 - precision_m: 0.6270 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.5283 - val_f1_m: 0.7575 - val_precision_m: 0.6121 - val_recall_m: 1.0000
Epoch 2/32
169/169 [==============================] - 1s 3ms/step - loss: 0.6932 - acc: 0.5019 - f1_m: 0.7689 - precision_m: 0.6267 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.4672 - val_f1_m: 0.7575 - val_precision_m: 0.6121 - val_recall_m: 1.0000
Epoch 3/32
169/169 [==============================] - 1s 3ms/step - loss: 0.6932 - acc: 0.4688 - f1_m: 0.7690 - precision_m: 0.6265 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.4435 - val_f1_m: 0.7575 - val_precision_m: 0.6121 - val_recall_m: 1.0000
Epoch 4/32
169/169 [==============================] - 1s 3ms/step - loss: 0.6932 - acc: 0.4496 - f1_m: 0.7690 - precision_m: 0.6269 - recall_m: 1.0000 - val_loss: 0.6931 - v

### Joint Model by Keras Concatenate

In [41]:
def create_concatenate_keras_model(models_arr):
    commonInput = Input(shape=[max_len])

    input_model_arr = []
    for model in models_arr: 
        outmodel = model(commonInput)
        input_model_arr.append(outmodel)
    
    mergedOut = Concatenate()(input_model_arr)

    mergedOut = Flatten()(mergedOut)    
    mergedOut = Dense(256, activation='relu')(mergedOut)
    mergedOut = Dropout(.5)(mergedOut)
    mergedOut = Dense(128, activation='relu')(mergedOut)
    mergedOut = Dropout(.35)(mergedOut)
    mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

    mergedModel = Model(commonInput, mergedOut)
    mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
    
    return mergedModel

In [ ]:
print('Joint model by concatenate, predicting deception')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model, rapport_model])
merged_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_deception), callbacks=[early_stop])

Joint model by concatenate, predicting deception
Epoch 1/15
85/85 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4996 - f1_m: 0.9745 - recall_m: 1.0000 - precision_m: 0.9504

In [ ]:
print('Joint model by concatenate, predicting rapport')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model])
merged_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_rapport), callbacks=[early_stop])